In [ ]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm 
import pandas as pd

In [ ]:
jobs_ = []
pages = 100

for i in tqdm(range(pages)) :
  
    session = requests.Session()
    adapter = requests.adapters.HTTPAdapter(max_retries=20)
    session.mount('https://', adapter)
    session.mount('http://', adapter)
    page=session.get(f'https://www.timesjobs.com/candidate/job-search.html?from=submit&actualTxtKeywords=data%20science&searchBy=0&rdoOperator=OR&searchType=personalizedSearch&luceneResultSize=25&postWeek=60&txtKeywords=data%20science&pDate=I&sequence={i}&startPage=31')
    soup = BeautifulSoup(page.text, 'html.parser')
    
    jobs = soup.findAll('li', class_='clearfix job-bx wht-shd-bx')
    jobs_links = [job.h2.a["href"] for job in jobs]
    
    for job in jobs_links :
        
        session = requests.Session()
        adapter = requests.adapters.HTTPAdapter(max_retries=20)
        session.mount('https://', adapter)
        session.mount('http://', adapter)
        ad=session.get(job)
        soup = BeautifulSoup(ad.text, 'html.parser')
        
        try :
            lists = soup.find('ul', {'class':'clearfix', 'id':'applyFlowHideDetails_1'}).findAll('li')
            job_info = [elt.text.strip().replace('\n',' ').replace('\r',' ').replace('\t',' ') for elt in lists]
            skills_tag = soup.find('div', class_ = 'jd-sec job-skills clearfix').div.find_all('span')
            skills = [skill.text for skill in skills_tag]
            location = soup.find('ul', class_ = 'top-jd-dtl clearfix').find_all('li')[2]['title']
        
        except :
            if not job_info :
                job_info = None
            elif not skills :
                skills = None
            elif not when_posted :
                when_posted = None
            elif not location :
                location = None
        
        details = {
            'Job info':job_info,
            'Skills': skills,
            'location': location
        }
        
        jobs_.append(details)
        
        

100%|██████████| 100/100 [58:05<00:00, 34.86s/it]


In [ ]:
df = pd.DataFrame(jobs_)

In [ ]:
df.shape

(2475, 3)

In [ ]:
df.head()

,Job info,Skills,location
0,[Job Function: IT Software : Software Products...,"[data analysis , sql , data science , analyst ...",Bengaluru/ Bangalore (Karnataka)
1,[Job Function: IT Software : Software Products...,"[data analysis , sql , data science , analyst ...",Bengaluru/ Bangalore (Karnataka)
2,[Job Function: IT Software : Software Products...,"[advanced analytics , data analytics , sas , c...",Bengaluru/ Bangalore (Karnataka)
3,[Job Function: IT Software : Software Products...,"[sql , data science , use cases , data analyti...","Mumbai (Maharashtra), Chennai (Tamil Nadu), Pu..."
4,[Job Function: IT Software : Software Products...,"[data mining , api , machine learning ]",Bengaluru/ Bangalore (Karnataka)


In [ ]:
df.to_csv('data_science.csv')